<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
Автор материала: аналитик-разработчик в команде Яндекс.Метрики Мария Мансурова. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Домашнее задание № 9
## <center> Анализ временных рядов
**Заполните код в клетках и выберите ответы в [веб-форме](https://docs.google.com/forms/d/1l3OtddjlKGGyl285bkzu4ulH5cXSnBdAkqR9VzB3Bgg/edit).**

In [12]:
import pandas as pd
import os


from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import pandas as pd

print(__version__) # need 1.9.0 or greater

init_notebook_mode(connected = True)




def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

2.3.0


In [13]:
def pltly_df ( df , title = ''):
    data  = []
    for column in df.columns :
        td

## Подготавливаем данные

Для начала скачаем данные в `dataframe`. Сегодня будем предсказывать просмотры wiki-страницы [Machine Learning](https://en.wikipedia.org/wiki/Machine_learning). Данные я скачала с помощью библиотеки [Wikipediatrend](https://www.r-bloggers.com/using-wikipediatrend/) для `R`.

In [14]:
df = pd.read_csv('data/wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [15]:
df.shape

(383, 7)

In [16]:
df.date = pd.to_datetime(df.date)

In [17]:
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [18]:
plotly_df(df.set_index('date')[['count']])

## Предсказание с помощью Facebook Prophet

Для начала построим предсказание с помощью простой библиотеки `Facebook Prophet`. Для того, чтобы посмотреть на качество модели, отбросим из обучающей выборки последние 30 дней.

In [19]:
from fbprophet import Prophet

In [20]:
predictions = 30

df = df[['date', 'count']]
df.columns = ['ds', 'y']
train_df = df[:-predictions].copy()

In [21]:
train_df

,ds,y
81,2015-01-01,1414
80,2015-01-02,1920
79,2015-01-03,1338
78,2015-01-04,1404
77,2015-01-05,2264
76,2015-01-06,2327
75,2015-01-07,2469
74,2015-01-08,2349
73,2015-01-09,2279
63,2015-01-10,1600


In [26]:
predictor = Prophet()
predictor.fit(train_df)
future = predictor.make_future_dataframe(periods=30)
forecast = predictor.predict(future)

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [30]:
forecast[forecast.ds=='2015-01-20'].trend.round(3)

19    2213.473
Name: trend, dtype: float64

**Вопрос 1:** Какое предсказание числа просмотров wiki-страницы на 20 января? Ответ округлите до целого числа.

Оценим качество предсказания по последним 30 точкам.

In [31]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))

In [32]:
cmp_df

,yhat,yhat_lower,yhat_upper,y
ds,,,,
2015-01-01,2486.413875,2080.547262,2877.284688,1414
2015-01-02,2169.403604,1789.065980,2572.468496,1920
2015-01-03,1303.410073,934.624246,1723.252441,1338
2015-01-04,1447.242252,1039.149993,1851.931359,1404
2015-01-05,2452.228667,2039.123304,2852.336291,2264
2015-01-06,2715.045070,2310.059938,3132.194154,2327
2015-01-07,2601.968389,2211.309671,3003.414763,2469
2015-01-08,2506.326342,2143.215811,2945.667081,2349
2015-01-09,2189.316070,1804.397661,2616.956556,2279


**Вопрос 2**: Какое получилось MAPE?

**Вопрос 3**: Какое получилось MAE?

In [34]:
from sklearn.metrics import mean_absolute_error

In [38]:
def mean_absolute_persentage_error (y, y_hat ) :
    return np.mean(np.abs((y - y_hat) / y)) * 100

In [39]:
import numpy as np

In [41]:
print ('MAPE : ' , mean_absolute_persentage_error(cmp_df.y , cmp_df.yhat))
print ('MAE : ' , mean_absolute_error(cmp_df.y , cmp_df.yhat))


MAPE :  10.584790236987924
MAE :  219.504795468


## Предсказываем с помощью ARIMA

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

**Вопрос 4:** Проверим стационарность ряда с помощью критерия Дики-Фулера. Является ли ряд стационарным? Какое значение p-value?

In [49]:
from statsmodels.tsa.stattools import adfuller

In [54]:
train_df = train_df.set_index('ds')

In [58]:
train_df.head()

,y
ds,
2015-01-01,1414
2015-01-02,1920
2015-01-03,1338
2015-01-04,1404
2015-01-05,2264


In [ ]:
train

In [57]:
adfuller(train_df.values)

ValueError: too many values to unpack (expected 2)

In [55]:
test = adfuller(train_df)
print ('adf: ', test[0] )
print ('p-value: ', test[1])
print ('Critical values: ', test[4])
if test[0]> test[4]['5%']: 
    print ('есть единичные корни, ряд не стационарен')
else:
    print( 'единичных корней нет, ряд стационарен')

ValueError: too many values to unpack (expected 2)

In [25]:
## ВАШ КОД для проверки стационарности ряда ##

**Вопрос 5**: Далее перейдем к построению модели SARIMAX (`sm.tsa.statespace.SARIMAX`). Модель c какими параметрами лучшая по `AIC`-критерию?

In [26]:
## ВАШ КОД для построения модели ##